In [39]:
from openai import OpenAI
import pandas as pd
import json
import ast
from dotenv import load_dotenv
import os

load_dotenv()

my_key = os.getenv("MY_API_KEY")

In [41]:
client = OpenAI(api_key=my_key)

In [17]:
MODEL = "gpt-4o-mini"

response = client.chat.completions.create(
    model=MODEL,
    messages=[
        {"role": "system", "content": "You are a helpful assistant that helps me synthesizing things."},
        {"role": "user", "content": "Hello! Could you help me synthesize a mock dataset formatted as a list of dictionaries with 1000 rows that represent a 'Twitch chat hate speech' dataset with 2 columns called Message where it is the message and 'Hate' where it is a 0/1 value to see if it's a hate speech or not. I want you to synthesize it and give me the list only, without anything else. Just give me in plain text, no codeblock needed"}
    ]
)

generated_content =  response.choices[0].message.content

In [18]:
print(type(generated_content))
print(generated_content)

<class 'str'>
[{'Message': 'I love this game!', 'Hate': 0},
 {'Message': 'You all are terrible at this!', 'Hate': 1},
 {'Message': 'This streamer is the best!', 'Hate': 0},
 {'Message': 'Shut up, you noob!', 'Hate': 1},
 {'Message': 'So much fun to watch!', 'Hate': 0},
 {'Message': 'Get lost, loser!', 'Hate': 1},
 {'Message': 'Nice play!', 'Hate': 0},
 {'Message': 'You should just quit!', 'Hate': 1},
 {'Message': 'Love the community here!', 'Hate': 0},
 {'Message': 'What a bunch of idiots!', 'Hate': 1},
 {'Message': 'GG everyone!', 'Hate': 0},
 {'Message': 'Stop wasting your time!', 'Hate': 1},
 {'Message': 'Great team effort!', 'Hate': 0},
 {'Message': 'You’re just a wannabe!', 'Hate': 1},
 {'Message': 'Can’t wait for the next match!', 'Hate': 0},
 {'Message': 'You all suck!', 'Hate': 1},
 {'Message': 'This is epic!', 'Hate': 0},
 {'Message': 'What a dumb thing to do!', 'Hate': 1},
 {'Message': 'So entertaining!', 'Hate': 0},
 {'Message': 'Why are you even here?', 'Hate': 1},
 {'Messa

In [20]:
data_list = ast.literal_eval(generated_content)

In [25]:
data_list

[{'Message': 'I love this game!', 'Hate': 0},
 {'Message': 'You all are terrible at this!', 'Hate': 1},
 {'Message': 'This streamer is the best!', 'Hate': 0},
 {'Message': 'Shut up, you noob!', 'Hate': 1},
 {'Message': 'So much fun to watch!', 'Hate': 0},
 {'Message': 'Get lost, loser!', 'Hate': 1},
 {'Message': 'Nice play!', 'Hate': 0},
 {'Message': 'You should just quit!', 'Hate': 1},
 {'Message': 'Love the community here!', 'Hate': 0},
 {'Message': 'What a bunch of idiots!', 'Hate': 1},
 {'Message': 'GG everyone!', 'Hate': 0},
 {'Message': 'Stop wasting your time!', 'Hate': 1},
 {'Message': 'Great team effort!', 'Hate': 0},
 {'Message': 'You’re just a wannabe!', 'Hate': 1},
 {'Message': 'Can’t wait for the next match!', 'Hate': 0},
 {'Message': 'You all suck!', 'Hate': 1},
 {'Message': 'This is epic!', 'Hate': 0},
 {'Message': 'What a dumb thing to do!', 'Hate': 1},
 {'Message': 'So entertaining!', 'Hate': 0},
 {'Message': 'Why are you even here?', 'Hate': 1},
 {'Message': 'That was

In [ ]:
res = json.dumps(data_list, indent=2, default=lambda x: x)
try:
    data = json.loads(res)
    print("Convert complete")
except Exception as e:
    print(f"Error making OpenAI API call: {e}")
    data = []

In [36]:
df = pd.DataFrame(data)
df.to_csv("synthesized_twitch_chats.csv", index=False)